In [40]:
import os
from google.colab import userdata

try:
  key = userdata.get("GROQ_API_KEY")
  os.environ["GROQ_API_KEY"]= key
  print("groq key has been uploaded successfully!")
  print(f"key length : {len(key)}")
except Exception as e:
  print(f"Exception : {e}")

groq key has been uploaded successfully!
key length : 56


In [4]:
# Upgrade pip slightly and install the stack in one careful go
!pip install -q --upgrade pip
!pip install -q -U \
    langchain==0.3.3 \
    langchain-core==0.3.3 \
    langchain-community==0.3.3 \
    langchain-groq==0.2.0 \
    chromadb==0.5.5 \
    gradio==4.44.0 \
    sentence-transformers \
    pandas \
    groq==0.11.0

# If Chroma complains about missing wheels later, add these (usually not needed)
!pip install -q --no-deps chroma-hnswlib==0.7.6

ERROR: Cannot install langchain-core==0.3.3 and langchain==0.3.3 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [5]:
import os
import pandas as pd
from google.colab import userdata

from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

inventory_data = """
product_id,product_name,category,quantity_in_stock,unit_price,last_updated
P001,Wireless Mouse,Electronics,142,24.99,2025-12-20
P002,USB-C Cable 2m,Electronics,315,8.50,2025-11-15
P003,Mechanical Keyboard,Electronics,38,89.00,2026-01-05
P004,HDMI Cable 1.5m,Electronics,87,12.99,2025-10-30
P005,Office Chair,Furniture,19,149.99,2026-02-01
P006,Desk Lamp,Furniture,64,34.50,2025-12-10
P007,Notebooks (Pack 5),Stationery,210,9.99,2026-01-25
P008,Blue Ballpoint Pens (Pack 10),Stationery,178,4.99,2025-09-18
P009,4K Monitor 27",Electronics,12,299.00,2026-02-03
P010,Laptop Stand,Accessories,53,22.50,2025-11-28
"""

with open("inventory.csv", "w") as f:
  f.write(inventory_data)

""" above is open a new file named inventory.csv
    to writing purpose and write data into it"""


' above is open a new file named inventory.csv\n    to writing purpose and write data into it'

In [6]:
!pip install -qU \
    langchain \
    langchain-community \
    langchain-text-splitters \
    langchain-groq \
    chromadb \
    sentence-transformers

In [1]:
!pip install -qU langchain langchain-community langchain-text-splitters langchain-core

In [6]:
!pip install -q langchain-text-splitters

In [2]:
!pip install langchain-groq -qU

In [2]:
# First, uninstall all existing langchain related packages to ensure a clean slate.
!pip uninstall -y langchain langchain-core langchain-community langchain-groq langchain-text-splitters chromadb sentence-transformers

Found existing installation: langchain 1.2.9
Uninstalling langchain-1.2.9:
  Successfully uninstalled langchain-1.2.9
Found existing installation: langchain-core 1.2.9
Uninstalling langchain-core-1.2.9:
  Successfully uninstalled langchain-core-1.2.9
Found existing installation: langchain-community 0.4.1
Uninstalling langchain-community-0.4.1:
  Successfully uninstalled langchain-community-0.4.1
Found existing installation: langchain-groq 1.1.2
Uninstalling langchain-groq-1.1.2:
  Successfully uninstalled langchain-groq-1.1.2
Found existing installation: langchain-text-splitters 1.1.0
Uninstalling langchain-text-splitters-1.1.0:
  Successfully uninstalled langchain-text-splitters-1.1.0
Found existing installation: chromadb 1.4.1
Uninstalling chromadb-1.4.1:
  Successfully uninstalled chromadb-1.4.1
Found existing installation: sentence-transformers 5.2.2
Uninstalling sentence-transformers-5.2.2:
  Successfully uninstalled sentence-transformers-5.2.2


In [6]:
df = pd.read_csv("inventory.csv")
print(f"sample inventory")
print(df.head(10))

sample inventory
  product_id                   product_name     category  quantity_in_stock  \
0       P001                 Wireless Mouse  Electronics                142   
1       P002                 USB-C Cable 2m  Electronics                315   
2       P003            Mechanical Keyboard  Electronics                 38   
3       P004                HDMI Cable 1.5m  Electronics                 87   
4       P005                   Office Chair    Furniture                 19   
5       P006                      Desk Lamp    Furniture                 64   
6       P007             Notebooks (Pack 5)   Stationery                210   
7       P008  Blue Ballpoint Pens (Pack 10)   Stationery                178   
8       P009                 4K Monitor 27"  Electronics                 12   
9       P010                   Laptop Stand  Accessories                 53   

   unit_price last_updated  
0       24.99   2025-12-20  
1        8.50   2025-11-15  
2       89.00   2026-01-05

In [7]:
documents = []

for _, row in df.iterrows():
  text = (f"Product ID : {row['product_id']}\n"
      f"Name : {row['product_name']}\n"
      f"Category : {row['category']}\n"
      f"Stock : {row['quantity_in_stock']}\n"
      f"Price : {row['unit_price']}\n"
      f"Last Updated : {row['last_updated']}\n"
      )
  documents.append(text)
print(f"\n created {len(documents)} documents chunks")


 created 10 documents chunks


In [8]:
import torch
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs = {"device" : "cuda" if torch.cuda.is_available() else "cpu"}
)

# model name and use gpu if available else cpu for model

/tmp/ipython-input-409101030.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [20]:
!pip install chromadb

  Using cached opentelemetry_sdk-1.39.1-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_api-1.39.1-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_semantic_conventions-0.60b1-py3-none-any.whl.metadata (2.4 kB)
Using cached opentelemetry_sdk-1.39.1-py3-none-any.whl (132 kB)
Using cached opentelemetry_api-1.39.1-py3-none-any.whl (66 kB)
Using cached opentelemetry_semantic_conventions-0.60b1-py3-none-any.whl (219 kB)
  Attempting uninstall: opentelemetry-api
    Found existing installation: opentelemetry-api 1.37.0
    Uninstalling opentelemetry-api-1.37.0:
      Successfully uninstalled opentelemetry-api-1.37.0
  Attempting uninstall: opentelemetry-semantic-conventions
    Found existing installation: opentelemetry-semantic-conventions 0.58b0
    Uninstalling opentelemetry-semantic-conventions-0.58b0:
      Successfully uninstalled opentelemetry-semantic-conventions-0.58b0
  Attempting uninstall: opentelemetry-sdk
    Found existing installation: opentelem

In [9]:
from langchain.vectorstores import Chroma
""" this chunk of code is to create embeddings using chroma
    stored in persistent by default in folder ./chroma"""
vectorstore = Chroma.from_texts(
    # we are working on texts here
    texts = documents,
    embedding = embedding_model,
    collection_name = "inventory_collection"
)

In [10]:
""" this code is for retriveing the top 4 simmilar
   embeddings or answers from database when question
   asked by user"""
retriever = vectorstore.as_retriever(
    search_type = "similarity",# simmilar rows
    search_kwargs = {"k" : 4} # get top 4 relavant rows
)


In [16]:
pip install -U groq langchain langchain-community langchain-groq httpx


  Using cached langchain-1.2.9-py3-none-any.whl.metadata (5.7 kB)
  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_groq-1.1.2-py3-none-any.whl.metadata (2.4 kB)
  Using cached langchain_core-1.2.9-py3-none-any.whl.metadata (4.4 kB)
  Using cached langchain_text_splitters-1.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached groq-0.37.1-py3-none-any.whl.metadata (16 kB)
Using cached langchain-1.2.9-py3-none-any.whl (111 kB)
Using cached langchain_core-1.2.9-py3-none-any.whl (496 kB)
Using cached langchain_community-0.4.1-py3-none-any.whl (2.5 MB)
Using cached langchain_text_splitters-1.1.0-py3-none-any.whl (34 kB)
Using cached langchain_groq-1.1.2-py3-none-any.whl (19 kB)
Using cached groq-0.37.1-py3-none-any.whl (137 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.147
    Uninstalling langsmith-0.1.147:
      Successfully uninstalled langsmith-0.1.147
  Attempting uninstall: groq
    Found existi

In [1]:
! pip uninstall -y groq langchain langchain-community langchain-groq httpx
! pip install groq==0.9.0
! pip install langchain==0.2.14
! pip install langchain-community==0.2.12
! pip install langchain-groq==0.1.9
! pip install httpx==0.27.0


Found existing installation: groq 0.9.0
Uninstalling groq-0.9.0:
  Successfully uninstalled groq-0.9.0
Found existing installation: httpx 0.28.1
Uninstalling httpx-0.28.1:
  Successfully uninstalled httpx-0.28.1
  Using cached groq-0.9.0-py3-none-any.whl.metadata (13 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
Using cached groq-0.9.0-py3-none-any.whl (103 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [groq]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.61.0 requires websockets<15.1.0,>=13.0.0, but you have websockets 12.0 which is incompatible.
  Using cached langchain-0.2.14-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_core-0.2.43-py3-none-any.whl.metadata (6.2 kB)
  Using cached langchain_text_splitters-0.2.4-py3-none-any.whl.metadata (2.3 kB)
  Usi

In [49]:
""" Its instruction tuned , low temperature which means
  it provides a less creative response focusing more on
  embeddings and hence it creates the answering model"""
import os
from langchain_groq import ChatGroq
llm = ChatGroq(
    model = "llama-3.1-8b-instant",
    temperature = 0.2, # less creative
    groq_api_key = key,
    max_tokens = 500
)

In [51]:
system_prompt = """You are an Inventory Management Assistant for a retail store.
Use ONLY the following pieces of retrieved inventory data to answer the question.
If the information is not present in the retrieved context, say "Sorry, I don't have that information in the current inventory records."

Keep answers concise, professional and include units/prices when relevant.
Do NOT make up numbers or products.

Retrieved inventory context:
{context}

Current date: February 2026
"""
# above is prompt is to prevent hallucination - the information must be within above context


In [52]:
prompt = ChatPromptTemplate([
    ("system", system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])
""" as per langchain model all conversion history must be inserted in
chat history ,
Human: input
assistant : answer"""

' as per langchain model all conversion history must be inserted in\nchat history ,\nHuman: input\nassistant : answer'

In [53]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

""" this says take the retrived documents and put into context
based on the prompt and then provide it to large learning model
to get back the answer in nlp"""



' this says take the retrived documents and put into context\nbased on the prompt and then provide it to large learning model\nto get back the answer in nlp'

In [81]:
store = {}

def get_session_history(session_id : str):
  if session_id not in store:
    store[session_id] = ChatMessageHistory()
  return store[session_id]


""" above code is to find the history is the id
not present in the store than create empty history
else return the chat history"""

' above code is to find the history is the id\nnot present in the store than create empty history\nelse return the chat history'

In [82]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key = "input",
    history_messages_key = "chat_history",
    output_messages_key = "answer"
)
"""
here it first loads the rag chain and updates the memory.
for this latest session_id it loads all the chat history
and stores in chat_history after the assistance answers
it stores the new conversation in id
"""

'\nhere it first loads the rag chain and updates the memory.\nfor this latest session_id it loads all the chat history\nand stores in chat_history after the assistance answers\nit stores the new conversation in id\n'

In [83]:
# gradio chat inference

def chat_inventory(message, history):
  session_id = "inventory_session"
  response = conversational_rag_chain.invoke(


    {"input":message},
    config = {"configurable" : {"session_id" : session_id}}
      )

  answer = response["answer"]
  return answer

"""
in config we have fixed session_id = "session_id" and message is
given as input here and it returns the answer which is
accessed from response and then returned to gradio
"""




'\nin config we have fixed session_id = "session_id" and message is\ngiven as input here and it returns the answer which is\naccessed from response and then returned to gradio\n'

In [86]:
import gradio as gr
demo = gr.ChatInterface(
    fn = chat_inventory,
    title = "Inventory management chatbot",
    description = "Ask about stock levels, products, prices, low stock items, etc.",
    cache_examples = False,
    retry_btn = None
)
demo.launch(debug = True)

"""
this is like creating chat ui history display
debug = True -> shows detailed logs
"""

/usr/local/lib/python3.12/dist-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://573556ed51202211a5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://573556ed51202211a5.gradio.live


'\nthis is like creating chat ui history display\ndebug = True -> shows detailed logs\n'